<a href="https://colab.research.google.com/github/Donalizasaji/LAB/blob/main/Dona_515_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow transformers tensorflow_datasets


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import BertTokenizer, TFBertForSequenceClassification

# Load the IMDB dataset
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

# Split the dataset into training and testing sets
train_data, test_data = dataset['train'], dataset['test']

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenization function
def encode_texts(texts, max_len=128):
    return tokenizer(texts, truncation=True, padding='max_length', max_length=max_len, return_tensors='tf')

# Tokenize and encode the data
def encode_data(data):
    texts = []
    labels = []
    for text, label in data:
        texts.append(text.numpy().decode('utf-8'))
        labels.append(label.numpy())
    encodings = encode_texts(texts)
    return tf.data.Dataset.from_tensor_slices((dict(encodings), labels))

# Apply the encoding
train_data = train_data.map(lambda x, y: tf.py_function(encode_data, inp=[x, y], Tout=(dict, tf.int64)))
test_data = test_data.map(lambda x, y: tf.py_function(encode_data, inp=[x, y], Tout=(dict, tf.int64)))

# Batch and prefetch
batch_size = 32
train_data = train_data.shuffle(10000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
test_data = test_data.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.DQUH1F_1.0.0/imdb_reviews-train.tfrecor…

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

TypeError: in user code:

    File "<ipython-input-2-d22ecb22513d>", line 29, in None  *
        lambda x, y: tf.py_function(encode_data, inp=[x, y], Tout=(dict, tf.int64))

    TypeError: Cannot convert the argument `type_value`: <class 'dict'> to a TensorFlow DType.


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import BertTokenizer, TFBertForSequenceClassification

# Load the IMDB dataset
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_data, test_data = dataset['train'], dataset['test']

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenization function
def encode_texts(texts, max_len=128):
    encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=max_len, return_tensors='tf')
    return encodings

# Process the data
def encode_data(text, label):
    encoding = encode_texts([text.numpy().decode('utf-8')])
    return {'input_ids': encoding['input_ids'][0], 'attention_mask': encoding['attention_mask'][0]}, label

def tf_encode_data(text, label):
    return tf.py_function(encode_data, inp=[text, label],
                          Tout=({'input_ids': tf.int32, 'attention_mask': tf.int32}, tf.int64))

# Apply the encoding
train_data = train_data.map(tf_encode_data)
test_data = test_data.map(tf_encode_data)

# Batch and prefetch
batch_size = 32
train_data = train_data.shuffle(10000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
test_data = test_data.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# Load the pre-trained BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
epochs = 3
model.fit(train_data, validation_data=test_data, epochs=epochs)

# Evaluate the model
loss, accuracy = model.evaluate(test_data)
print(f'Loss: {loss}, Accuracy: {accuracy}')


TypeError: in user code:

    File "<ipython-input-3-9a15b9f72841>", line 23, in tf_encode_data  *
        return tf.py_function(encode_data, inp=[text, label],

    TypeError: Cannot convert the argument `type_value`: {'input_ids': tf.int32, 'attention_mask': tf.int32} to a TensorFlow DType.
